In [ ]:
SOURCE = "H:\My Drive\centroset\set30.5"

In [ ]:
abm = "H:\\My Drive\\anbev-model.keras"

In [ ]:
import glob, random, os
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

## generate data

In [ ]:
import numpy as np
MAXSHAPE = 30
def fill40(arr: np.array):
    padded = np.zeros((MAXSHAPE, MAXSHAPE), dtype=arr.dtype)
    padded[:arr.shape[0], :arr.shape[1]] = arr[:, :MAXSHAPE]
    return padded

In [ ]:
file_list = glob.glob(SOURCE+'\*.csv')
random.shuffle(file_list)
X = []
for csv in file_list:
    print(f"loading {csv}")
    df = pd.read_csv(csv, index_col="Unnamed: 0")
    X.append(fill40(df.values))

y = [0 if f.startswith("normal") else 1 for f in map(os.path.basename, file_list)]

In [ ]:
X_train = np.array(X[:40])
y_train = np.array(y[:40])
X_val = np.array(X[40:])
y_val = np.array(y[40:])

## Building the Model from Scratch

But before we continue, let's start defining the model:

Step 1 will be to import tensorflow.

In [1]:
import tensorflow as tf
import numpy as np
#from itertools import cycle

#from sklearn import svm, datasets
# from sklearn.metrics import roc_curve, auc
#from sklearn.model_selection import train_test_split
#from sklearn.preprocessing import label_binarize
#from sklearn.multiclass import OneVsRestClassifier
#from scipy import interp
#from sklearn.metrics import roc_auc_score

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape = (30, 30)),
                                    tf.keras.layers.Dense(10, activation=tf.nn.relu),
                                    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)])

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation=tf.nn.relu, input_shape=(30, 30)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(input_shape = (30, 30)),
    tf.keras.layers.Dense(10, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)
    ])

The model.summary() method call prints a summary of the NN

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 900)               0         
                                                                 
 dense (Dense)               (None, 10)                9010      
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 9021 (35.24 KB)
Trainable params: 9021 (35.24 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


The "output shape" column shows the transformation of the dimensions of each layer as a result of the convolution and max pooling - convolution will reduce the layer size by a bit due to padding, and max pooling will halve the output size.

Next, we'll configure the specifications for model training. We will train our model with the `binary_crossentropy` loss. We will use the `Adam` optimizer. [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) is a sensible optimization algorithm because it automates learning-rate tuning for us (alternatively, we could also use [RMSProp](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) or [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad) for similar results). We will add accuracy to `metrics` so that the model will monitor accuracy during training

In [ ]:
model.compile(optimizer = tf.optimizers.Adam(),
              loss = 'binary_crossentropy',
              metrics=['accuracy'])

### Training
Let's train for 15 epochs.

Note that steps_per_epoch was set along with batch_size in ImageDataGenerator so that steps_per_epoch * batch_size = total # of images. For example, for training, 8 * 120 = 960, just under our total of 999 images.

Notice that as we train, our validation accuracy never exceeds training accuracy, which is a good thing. Our model won't work better on unseen images than seen images.

In [ ]:
# history = model.fit(
#       train_generator,
#       steps_per_epoch=8,
#       epochs=15,
#       verbose=1,
#       validation_data = validation_generator,
#       validation_steps=8)

history = model.fit(x=X_train,
      y=y_train,
      steps_per_epoch=4,
      batch_size=15,
      epochs=5,
      verbose=1,
      validation_data=(X_val, y_val),
      shuffle=True
      # validation_data = validation_generator,
      )

In [ ]:
model.save('model.keras')

## Accuracy, ROC Curve, and AUC

Let's evaluate the accuracy of our model:

In [ ]:
model.evaluate(X_val, y_val, verbose=1)

Now, let's calculate our ROC curve and plot it.

First, let's make predictions on our validation set. When using generators to make predictions, we must first turn off shuffle (as we did when we created validation_generator) and reset the generator:

In [ ]:
# load model.keras
model = tf.keras.models.load_model('model.keras')

In [ ]:
preds = model.predict(X_val,
                      verbose=1)

In [ ]:
np.hstack(preds)[6]

In [ ]:
# filter np array that > 0


In [ ]:
y_val

In [ ]:
i = random.randint(0, len(X))
pred = model.predict(np.array([X[i]]))
pred = pred.round().astype(int)[0,0]
print(f"index: {i} pred: {pred}, label: {y[i]}")

To create the ROC curve and AUC, we'll need to compute the false-positive rate and the true-positive rate:

In [ ]:
fpr, tpr, _ = roc_curve(y_val, preds)

In [ ]:
roc_auc = auc(fpr, tpr)

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

The ROC curve is a probability curve plotting the true-positive rate (TPR) against the false-positive rate (FPR). In this curve, the diagonal line is the curve for random guessing, e.g. coin flipping, so the ROC curve above shows that our model does better than chance at classifying between dandelions and grass. Not bad!

Similarly, the AUC (area under curve), as shown in the legend above, measures how much our model is capable of distinguishing between our two classes, dandelions and grass. The higher the AUC, the better our model is at classification. It is also used to compare different models, which I will do in future tutorials when I present how to build an image classifier using Convolutional Neural Networks and transfer learning with ResNet!

## Making Predictions

Now, let's use the model to make predictions! Upload an image to see if it's a dandelion or grass.

In [ ]:
model

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(200, 200))
  x = image.img_to_array(img)
  plt.imshow(x/255.)
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]<0.5:
    print(fn + " is a dandelion")
  else:
    print(fn + " is a grass")


## Clean Up
Run the following cell to terminate the kernel and free memory resources:

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)

In [ ]:
round(0.5)

In [ ]:
import IPython
display(IPython.display.Javascript('window.location.reload()'))